# Preliminary Pipeline

In [1]:
import pandas as pd
import numpy as np

## Cases data

In [2]:
num_records = 10_000_000

In [3]:
df_cases = pd.read_csv(
    './../data/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv',
    dtype='string',
    nrows=num_records)

In [4]:
df_cases = df_cases.drop(columns=['state_fips_code'])

In [5]:
df_cases['case_positive_specimen_interval'] = pd.to_numeric(
    df_cases['case_positive_specimen_interval'])
df_cases['case_onset_interval'] = pd.to_numeric(
    df_cases['case_onset_interval'])

In [6]:
df_cases

,case_month,res_state,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-08,KY,GRAYSON,21085,0 - 17 years,<NA>,<NA>,<NA>,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,<NA>
1,2021-01,NC,MARTIN,37117,0 - 17 years,<NA>,<NA>,<NA>,0.0,NaN,Missing,Unknown,Laboratory-confirmed case,Asymptomatic,No,Unknown,No,<NA>
2,2021-03,VT,FRANKLIN,50011,18 to 49 years,<NA>,<NA>,<NA>,0.0,0.0,Missing,Yes,Laboratory-confirmed case,Symptomatic,No,Missing,No,Yes
3,2020-09,GA,CATOOSA,13047,0 - 17 years,Female,Unknown,Missing,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,<NA>
4,2021-04,PA,CLINTON,42035,0 - 17 years,Female,<NA>,<NA>,0.0,0.0,Missing,Yes,Laboratory-confirmed case,Symptomatic,No,No,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,2021-02,FL,ST. LUCIE,12111,65+ years,Male,White,Non-Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Yes,<NA>
9999996,2020-12,UT,WASHINGTON,49053,65+ years,Male,White,Non-Hispanic/Latino,0.0,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,Yes,Yes
9999997,2020-10,IL,KENDALL,17093,50 to 64 years,Female,<NA>,<NA>,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,<NA>
9999998,2020-07,NY,NASSAU,36059,0 - 17 years,Male,Missing,Unknown,0.0,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,<NA>


## Counties data

In [7]:
use_columns = ['STATE', 'COUNTY', 'County FIPS', 'Low Income Area (LIA) County SAIPE- Score', 'Tribal Community\n(1 if yes)', 'Rural']
column_names = ['state', 'county', 'county_fips', 'low_income_score', 'tribal', 'rural']

In [8]:
df_counties = pd.read_csv(
    './../data/COVID-19_Community_Vulnerability_Crosswalk_-_Crosswalk_by_Census_Tract.csv',
    usecols=use_columns,
    dtype='string')

In [9]:
df_counties.columns = column_names

In [10]:
df_counties = df_counties.astype({'low_income_score': 'int64', 'rural': 'int64'})

In [11]:
df_counties['tribal'] = np.where(df_counties['tribal'] == 'Non-Tribal', 0, 1)
df_counties['ctract_count'] = 1

In [12]:
df_counties = df_counties.groupby(['state', 'county', 'county_fips']).agg(
    {'low_income_score': 'mean', 'tribal': 'sum', 'rural': 'sum', 'ctract_count': 'sum'}).reset_index()

In [13]:
df_counties['perc_tribal_ct'] = df_counties['tribal'] / df_counties['ctract_count']
df_counties['perc_rural_ct'] = df_counties['rural'] / df_counties['ctract_count']

In [14]:
df_counties = df_counties.drop(columns=['state', 'county', 'tribal', 'rural', 'ctract_count'])

In [15]:
df_counties

,county_fips,low_income_score,perc_tribal_ct,perc_rural_ct
0,01001,0,0.0,0.250000
1,01003,0,0.0,0.290323
2,01005,15,0.0,1.000000
3,01007,15,0.0,1.000000
4,01009,7,0.0,0.888889
...,...,...,...,...
3136,56037,0,0.0,0.250000
3137,56039,0,0.0,1.000000
3138,56041,0,0.0,1.000000
3139,56043,0,0.0,1.000000


## Joining

In [16]:
df = df_cases.merge(
    df_counties, how='left', left_on='county_fips_code', right_on='county_fips')

In [17]:
df = df.drop(columns=['county_fips'])

In [18]:
df['res_county'] = df['res_county'] + ', ' + df['res_state']

In [19]:
df

,case_month,res_state,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,...,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn,low_income_score,perc_tribal_ct,perc_rural_ct
0,2020-08,KY,"GRAYSON, KY",21085,0 - 17 years,<NA>,<NA>,<NA>,NaN,0.0,...,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,<NA>,15.0,0.000000,1.000000
1,2021-01,NC,"MARTIN, NC",37117,0 - 17 years,<NA>,<NA>,<NA>,0.0,NaN,...,Unknown,Laboratory-confirmed case,Asymptomatic,No,Unknown,No,<NA>,15.0,0.000000,1.000000
2,2021-03,VT,"FRANKLIN, VT",50011,18 to 49 years,<NA>,<NA>,<NA>,0.0,0.0,...,Yes,Laboratory-confirmed case,Symptomatic,No,Missing,No,Yes,0.0,0.000000,1.000000
3,2020-09,GA,"CATOOSA, GA",13047,0 - 17 years,Female,Unknown,Missing,NaN,NaN,...,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,<NA>,0.0,0.000000,0.090909
4,2021-04,PA,"CLINTON, PA",42035,0 - 17 years,Female,<NA>,<NA>,0.0,0.0,...,Yes,Laboratory-confirmed case,Symptomatic,No,No,<NA>,<NA>,7.0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,2021-02,FL,"ST. LUCIE, FL",12111,65+ years,Male,White,Non-Hispanic/Latino,NaN,NaN,...,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Yes,<NA>,0.0,0.022727,0.000000
9999996,2020-12,UT,"WASHINGTON, UT",49053,65+ years,Male,White,Non-Hispanic/Latino,0.0,0.0,...,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,Yes,Yes,0.0,0.142857,0.238095
9999997,2020-10,IL,"KENDALL, IL",17093,50 to 64 years,Female,<NA>,<NA>,NaN,0.0,...,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,<NA>,0.0,0.000000,0.000000
9999998,2020-07,NY,"NASSAU, NY",36059,0 - 17 years,Male,Missing,Unknown,0.0,NaN,...,Missing,Probable Case,Missing,Missing,Missing,Missing,<NA>,0.0,0.000000,0.000000


In [20]:
del df_cases
del df_counties

In [21]:
%whos

Variable       Type         Data/Info
-------------------------------------
column_names   list         n=6
df             DataFrame            case_month res_st<...>000000 rows x 21 columns]
np             module       <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
num_records    int          10000000
pd             module       <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
use_columns    list         n=6


## Yes/No to death_yn column

In [22]:
df_keep = df[~df['death_yn'].isna()]

In [23]:
df_keep = df_keep[(df_keep['death_yn']=='Yes') | (df_keep['death_yn']=='No')]

In [24]:
df_keep.reset_index(drop=True, inplace=True)

In [25]:
df_keep

,case_month,res_state,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,...,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn,low_income_score,perc_tribal_ct,perc_rural_ct
0,2020-08,KY,"GRAYSON, KY",21085,0 - 17 years,<NA>,<NA>,<NA>,NaN,0.0,...,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,<NA>,15.0,0.000000,1.000000
1,2021-01,NC,"MARTIN, NC",37117,0 - 17 years,<NA>,<NA>,<NA>,0.0,NaN,...,Unknown,Laboratory-confirmed case,Asymptomatic,No,Unknown,No,<NA>,15.0,0.000000,1.000000
2,2021-03,VT,"FRANKLIN, VT",50011,18 to 49 years,<NA>,<NA>,<NA>,0.0,0.0,...,Yes,Laboratory-confirmed case,Symptomatic,No,Missing,No,Yes,0.0,0.000000,1.000000
3,2021-04,OR,"CROOK, OR",41013,0 - 17 years,Female,<NA>,<NA>,NaN,0.0,...,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,<NA>,0.0,0.000000,1.000000
4,2020-07,OK,"MUSKOGEE, OK",40101,0 - 17 years,Female,<NA>,<NA>,NaN,NaN,...,Missing,Laboratory-confirmed case,Unknown,No,Missing,No,<NA>,15.0,1.000000,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4413808,2020-11,NY,"ERIE, NY",36029,0 - 17 years,Female,White,Non-Hispanic/Latino,0.0,NaN,...,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,<NA>,0.0,0.033755,0.075949
4413809,2021-04,NJ,"BERGEN, NJ",34003,18 to 49 years,Female,White,Non-Hispanic/Latino,NaN,NaN,...,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,<NA>,0.0,0.000000,0.000000
4413810,2020-10,OR,"MULTNOMAH, OR",41051,18 to 49 years,Female,White,Non-Hispanic/Latino,NaN,0.0,...,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,<NA>,0.0,0.000000,0.000000
4413811,2021-02,FL,"ST. LUCIE, FL",12111,65+ years,Male,White,Non-Hispanic/Latino,NaN,NaN,...,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Yes,<NA>,0.0,0.022727,0.000000


In [26]:
del df

In [27]:
%whos

Variable       Type         Data/Info
-------------------------------------
column_names   list         n=6
df_keep        DataFrame            case_month res_st<...>413813 rows x 21 columns]
np             module       <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
num_records    int          10000000
pd             module       <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
use_columns    list         n=6


## Removing missing values

Removing all rows that have a missing, unkown, or NaN in exposure, ICU, or underlying conditions columns.

In [28]:
remove = df_keep['underlying_conditions_yn'].isin(['Missing', 'Unknown', pd.NA]) | df_keep['icu_yn'].isin(['Missing', 'Unknown', pd.NA]) | df_keep['exposure_yn'].isin(['Missing', 'Unknown', pd.NA])

In [29]:
f'Number of remaining rows: {df_keep[~remove].shape[0]:,}'

'Number of remaining rows: 101,672'

Removing all rows that have NaN, missing, or unknown in any column

In [30]:
remove = ((df_keep == 'Missing') | (df_keep == 'Unknown') | df_keep.isna()).any(axis=1)

In [31]:
f'Number of remaining rows: {df_keep[~remove].shape[0]:,}'

'Number of remaining rows: 22,706'